In [14]:
# !pip install beautifulsoup4
!pip install pymongo

In [15]:
from bs4 import BeautifulSoup
import unicodedata
import requests
from pymongo import MongoClient

In [16]:
def limpiar_unicode(texto):
    if texto:
        return unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('ascii')
    return texto


In [17]:
def obtener_precio_actual(soup):
    try:
        precio_actual = soup.find('span', {'class': 'a-price-whole'}).get_text(strip=True)
        precio_decimal = soup.find('span', {'class': 'a-price-fraction'}).get_text(strip=True)
        precio_actual = precio_actual + '.' + precio_decimal + ' €'
        return limpiar_unicode(precio_actual)
    except:
        return 'No disponible'

In [18]:
def obtener_precio_anterior(soup):
    try:
        precio_anterior = soup.find('span', {'class': 'a-price a-text-price'}).find('span', {'class': 'a-offscreen'}).get_text(strip=True)
        return limpiar_unicode(precio_anterior)
    except:
        return 'No disponible'


In [19]:
def obtener_detalles_tecnicos(soup):
    detalles_tecnicos = {}
    try:
        tabla_detalles = soup.find('table', {'id': 'productDetails_techSpec_section_1'})
        if tabla_detalles:
            filas = tabla_detalles.find_all('tr')
            for fila in filas:
                clave = fila.find('th').get_text(strip=True)
                valor = fila.find('td').get_text(strip=True)
                detalles_tecnicos[limpiar_unicode(clave)] = limpiar_unicode(valor)
        else:
            detalles_tecnicos = 'No disponible'
    except:
        detalles_tecnicos = 'No disponible'
    return detalles_tecnicos


In [20]:
def obtener_soup(api_key, product_id):
    url = 'https://www.amazon.es/dp/' + product_id
    params = {
        'api_key': api_key,
        'url': url,
        'country_code': 'ES'
    }

    response = requests.get('https://api.scraperapi.com/', params=params)

    if response.status_code == 200:
        return BeautifulSoup(response.text, 'html.parser')
    else:
        print(f'Error: {response.status_code}')
        return None

In [21]:
def insertar_en_mongodb(datos, db_name, collection_name, url = "mongodb://localhost:27017/"):
    # Conectar a la base de datos local de MongoDB (si usas un MongoDB remoto, cambia la URL de conexión)
    cliente = MongoClient(url)
    db = cliente[db_name]
    coleccion = db[collection_name]
    
    # Insertar los datos en la colección
    resultado = coleccion.insert_one(datos)
    print(f'Documento insertado con el ID: {resultado.inserted_id}')


In [22]:
def obtener_datos_producto(api_key, product_id):
    soup = obtener_soup(api_key, product_id)
    
    if soup:
        precio_actual = obtener_precio_actual(soup)
        precio_anterior = obtener_precio_anterior(soup)
        detalles_tecnicos = obtener_detalles_tecnicos(soup)

        resultado = {
            'precio_actual': precio_actual,
            'precio_anterior': precio_anterior,
            **detalles_tecnicos
        }
        return resultado

In [23]:
# API_KEY = '83bc7b5f85e1d83b695e0d39816ebb77'
# PRODUCT_ID = 'B0DLNRXQMV'

# detalles = obtener_datos_producto(API_KEY, PRODUCT_ID)
# print(detalles)
# insertar_en_mongodb(detalles, "PIA", "simple_products")

In [24]:
import json

# Cargar los IDs desde el archivo JSON
with open('../ids.json', 'r') as file:
    product_ids = json.load(file)

# API_KEY
API_KEY = '83bc7b5f85e1d83b695e0d39816ebb77'

# Iterar sobre los IDs de producto
for product_id in product_ids:
    detalles = obtener_datos_producto(API_KEY, product_id)
    print(detalles)
    insertar_en_mongodb(detalles, "PIA", "simple_products")


{'precio_actual': '109,.99 ', 'precio_anterior': '149,99', 'OS': 'Android', 'Peso del producto': '190 g', 'Dimensiones del producto': '16,8 x 0,83 x 7,67 cm; 190 g', 'Pilas': '1 AA necesaria(s), incluida(s)', 'Numero de modelo del producto': 'T509K3', 'Tecnologia de conectividad': 'Bluetooth, USB, NFC, Wifi', 'GPS': 'Verdadero', 'Caracteristicas especiales': 'Support OS upgrade to Android 16, Dual SIM, Expandable Memory, Octa Core Processor, Glass type: 2.5D glass(NXTPAPER: +AG)', 'Otras caracteristicas de la pantalla': 'Inalambrico', 'Entrada de interfaz humana': 'Microfono, Pantalla Tactil', 'Otras funciones de la camara': 'Trasera, Frontal', 'Conector de audio': '3.5 mm', 'Tamano': 'Touchscreen-Handy', 'Color': 'Space Gray', 'Potencia nominal de la bateria': '5010 Modificador desconocido', 'Tiempo de conversacion telefonica': '38 Horas', 'Que hay en la caja?': 'SIM Tray Ejector, Smartphone, USB Cable'}
Documento insertado con el ID: 67ad1e21d768004a42ff4768
{'precio_actual': '99,.99

TypeError: 'str' object is not a mapping